In AutoML.Net, [`SearchSpace`](https://docs.microsoft.com/en-us/dotnet/api/microsoft.ml.searchspace.searchspace?view=ml-dotnet-preview) defines the hyper-parameter searching range of a sweepable pipeline or a sweepable estimator and [`Parameter`](https://docs.microsoft.com/en-us/dotnet/api/microsoft.ml.searchspace.parameter?view=ml-dotnet-preview) is the sampled result from a `SearchSpace`, which can be used to restore an estimator or a pipeline.

`SearchSpace` has a great impact on automl performance. In theory, it decides the upper-bound of an automl model. A larger `SearchSpace` usually means an increasing potential of finding a better model. In practice though, there's also a trade-off and a larger `SearchSpace` doesn't always to be better considering the increasing searching complexity and higher training cost.

AutoML.Net provides default `SearchSpace` for almost all available ML.Net trainers, which is well-tuned on hundreds of dataset and should be good enough in 99% situation.  However, you can also provide your own search space if the default one doesn't meet your request.

In this notebook, we will go through the following topics on search space
- default search space for mlnet trainers.
- how to customize search space
- how to sample from search space

# Install nuget dependency and import using statement

In [ ]:
// install dependencies and import using statement
#i "nuget:https://pkgs.dev.azure.com/dnceng/public/_packaging/MachineLearning/nuget/v3/index.json"
#r "nuget: Plotly.NET.Interactive, 3.0.2"
#r "nuget: Plotly.NET.CSharp, 0.0.1"

// make sure you are using Microsoft.ML.AutoML later than 0.20.0.
#r "nuget: Microsoft.ML.AutoML, 0.20.0-preview.22356.1"
#r "nuget: Microsoft.Data.Analysis, 0.20.0-preview.22356.1"

Restore sources https://pkgs.dev.azure.com/dnceng/public/_packaging/MachineLearning/nuget/v3/index.json Installed Packages Microsoft.Data.Analysis, 0.20.0-preview.22356.1 Microsoft.ML.AutoML, 0.20.0-preview.22356.1 Plotly.NET.CSharp, 0.0.1 Plotly.NET.Interactive, 3.0.2

Loading extensions from `Microsoft.ML.AutoML.Interactive.dll`

Loading extensions from `Microsoft.Data.Analysis.Interactive.dll`

Loading extensions from `Plotly.NET.Interactive.dll`

In [ ]:
// Import usings.
using Microsoft.Data.Analysis;
using System;
using System.IO;
using Microsoft.ML;
using Microsoft.ML.AutoML;
using Microsoft.ML.Data;
using Microsoft.ML.SearchSpace;
using Newtonsoft.Json;
using Microsoft.ML.AutoML.CodeGen;
using Microsoft.ML.SearchSpace.Option;

# Default search space for ml.net trainer
AutoML.Net comes with a series of default search space for most of ml.net trainers. You can check it under `Microsoft.ML.AutoML.CodeGen` namespace. The following code shows the default search space for LightGbm.

The default search space will be used in `AutoFeaturizer` and `AutoTrainer` api, which provides an easy way to create sweepable estimators.

In [ ]:
var lgbmSearchSpace = new SearchSpace<LgbmOption>();

// the default option for NumberOfLeaves is (4, 32768, logBase: true)
// this might be too large but you can refine it if necessary
lgbmSearchSpace["NumberOfLeaves"] = new UniformIntOption(4, 1024, true, 4);

// pass lgbmSearchSpace in AutoTrainer
var context = new MLContext();
var pipeline = context.Auto().BinaryClassification(lgbmSearchSpace: lgbmSearchSpace);

JsonConvert.SerializeObject(lgbmSearchSpace, Formatting.Indented)

{
  "NumberOfLeaves": {
    "Min": 4.0,
    "Max": 1024.0,
    "LogBase": true,
    "FeatureSpaceDim": 1,
    "Step": [
      null
    ],
    "Default": [
      0.0
    ]
  },
  "MinimumExampleCountPerLeaf": {
    "Min": 20.0,
    "Max": 1024.0,
    "LogBase": true,
    "FeatureSpaceDim": 1,
    "Step": [
      null
    ],
    "Default": [
      0.0
    ]
  },
  "LearningRate": {
    "Min": 2E-10,
    "Max": 1.0,
    "LogBase": true,
    "FeatureSpaceDim": 1,
    "Step": [
      null
    ],
    "Default": [
      1.0
    ]
  },
  "NumberOfTrees": {
    "Min": 4.0,
    "Max": 32768.0,
    "LogBase": true,
    "FeatureSpaceDim": 1,
    "Step": [
      null
    ],
    "Default": [
      0.0
    ]
  },
  "SubsampleFraction": {
    "Min": 2E-10,
    "Max": 1.0,
    "LogBase": true,
    "FeatureSpaceDim": 1,
    "Step": [
      null
    ],
    "Default": [
      1.0
    ]
  },
  "MaximumBinCountPerFeature": {
    "Min": 8.0,
    "Max": 1024.0,
    "LogBase": true,
    "FeatureSpaceDim": 1,
 

# Create `SearchSpace` from scratch.
The following code shows how to create a `SearchSpace` that contains numeric, choice and nested options, where
- numeric option is an option that is numeric type, like float, double, int...
- choice option is an option that is descrete, like string or boolean.
- nested option is an option that itself is also a search space.


In [ ]:
using Microsoft.ML.SearchSpace.Option;

var searchSpace = new SearchSpace();

// numeric options
searchSpace["IntOption"] = new UniformIntOption(-10, 10, false, 0);
searchSpace["SingleOption"] = new UniformSingleOption(1, 10, true, 1);
searchSpace["DoubleOption"] = new UniformDoubleOption(-10, 10, false, 0);

// choice options
searchSpace["BoolOption"] = new ChoiceOption(true, false);
searchSpace["StrOption"] = new ChoiceOption("a", "b", "c");

// nest options
var nestedSearchSpace = new SearchSpace();
nestedSearchSpace["IntOption"] = new UniformIntOption(-10, 10, false, 0);
searchSpace["Nest"] = nestedSearchSpace;

// pretty print search space
JsonConvert.SerializeObject(searchSpace, Formatting.Indented)

{
  "IntOption": {
    "Min": -10.0,
    "Max": 10.0,
    "LogBase": false,
    "FeatureSpaceDim": 1,
    "Step": [
      null
    ],
    "Default": [
      0.5
    ]
  },
  "SingleOption": {
    "Min": 1.0,
    "Max": 10.0,
    "LogBase": true,
    "FeatureSpaceDim": 1,
    "Step": [
      null
    ],
    "Default": [
      0.0
    ]
  },
  "DoubleOption": {
    "Min": -10.0,
    "Max": 10.0,
    "LogBase": false,
    "FeatureSpaceDim": 1,
    "Step": [
      null
    ],
    "Default": [
      0.5
    ]
  },
  "BoolOption": {
    "Choices": [
      false,
      true
    ],
    "FeatureSpaceDim": 1,
    "Step": [
      2
    ],
    "Default": [
      0.0
    ]
  },
  "StrOption": {
    "Choices": [
      "a",
      "b",
      "c"
    ],
    "FeatureSpaceDim": 1,
    "Step": [
      3
    ],
    "Default": [
      0.0
    ]
  },
  "Nest": {
    "IntOption": {
      "Min": -10.0,
      "Max": 10.0,
      "LogBase": false,
      "FeatureSpaceDim": 1,
      "Step": [
        null
      ],


# create `SearchSpace<T>` which casts sampled parameter to a concrete type.
AutoML allows you to define `SearchSpace` with class type, in which case it checks class property and build up search space using those properties with option attributions. The following code shows how to create an identical search space from above using except with a class type.

`SearchSpace<T>`, comparing with `SearchSpace`, will also cast sampled parameter to a concrete type `T` instead of `Parameter`, which saves the effort of getting specific hyper-parameter value from `Parameter`.

In [ ]:
public class MyParameter
{
    [Range((int)-10, 10, 0, false)]
    public int IntOption {get; set;}

    [Range(1f, 10f, 1f, true)]
    public float SingleOption {get; set;}

    [Range(-10, 10, false)]
    public double DoubleOption {get; set;}

    [BooleanChoice]
    public bool BoolOption {get; set;}

    [Choice("a", "b", "c")]
    public string StrOption {get; set;}

    [NestOption]
    public NestParameter Nest {get; set;}
}

public class NestParameter
{
    [Range((int)-10, 10, 0, false)]
    public int IntOption {get; set;}
}

var searchSpace = new SearchSpace<MyParameter>();

// pretty print search space
JsonConvert.SerializeObject(searchSpace, Formatting.Indented)

{
  "IntOption": {
    "Min": -10.0,
    "Max": 10.0,
    "LogBase": false,
    "FeatureSpaceDim": 1,
    "Step": [
      null
    ],
    "Default": [
      0.5
    ]
  },
  "SingleOption": {
    "Min": 1.0,
    "Max": 10.0,
    "LogBase": true,
    "FeatureSpaceDim": 1,
    "Step": [
      null
    ],
    "Default": [
      0.0
    ]
  },
  "DoubleOption": {
    "Min": -10.0,
    "Max": 10.0,
    "LogBase": false,
    "FeatureSpaceDim": 1,
    "Step": [
      null
    ],
    "Default": [
      0.0
    ]
  },
  "BoolOption": {
    "Choices": [
      false,
      true
    ],
    "FeatureSpaceDim": 1,
    "Step": [
      2
    ],
    "Default": [
      0.0
    ]
  },
  "StrOption": {
    "Choices": [
      "a",
      "b",
      "c"
    ],
    "FeatureSpaceDim": 1,
    "Step": [
      3
    ],
    "Default": [
      0.0
    ]
  },
  "Nest": {
    "IntOption": {
      "Min": -10.0,
      "Max": 10.0,
      "LogBase": false,
      "FeatureSpaceDim": 1,
      "Step": [
        null
      ],


# Sampling `Parameter` from `SearchSpace`
In AutoML.Net, `SearchSpace` is associated with an `n`-dimension vector which is called `feature space`. And tuner, instead of performing sampling on original options in a `SearchSpace`, sampling from that `feature space` instead. The sampling result will then mapping back to options.

The following code shows the sampling process above. It creates an `n`-d random vector and uses that vector to sample from search space.

In [ ]:
var rnd = new Random();
SearchSpace searchSpace = new SearchSpace<MyParameter>();

foreach(var i in Enumerable.Range(0, 3))
{
    var dim = searchSpace.FeatureSpaceDim;
    var featureVector = Enumerable.Range(0, dim).Select(x => rnd.NextDouble()).ToArray();
    var parameter = searchSpace.SampleFromFeatureSpace(featureVector);

    // use Parameter.AsType<T> to map parameter to a concrete type.
    var myParameter = parameter.AsType<MyParameter>();
    var json = JsonConvert.SerializeObject(myParameter);

    Console.WriteLine($"{i} - {json}");

    // equivalant to
    var intOption = parameter["IntOption"].AsType<int>();
    var singleOption = parameter["SingleOption"].AsType<float>();
    var doubleOption = parameter["DoubleOption"].AsType<double>();
    var strOption = parameter["StrOption"].AsType<string>();
    var nest = parameter["Nest"].AsType<NestParameter>();

    myParameter = new MyParameter()
    {
        IntOption = intOption,
        SingleOption = singleOption,
        DoubleOption = doubleOption,
        StrOption = strOption,
        Nest = nest,
    };
    json = JsonConvert.SerializeObject(myParameter);

    Console.WriteLine($"{i} - {json}");
}

0 - {"IntOption":5,"SingleOption":1.3184657,"DoubleOption":2.0,"BoolOption":true,"StrOption":"b","Nest":{"IntOption":4}}
0 - {"IntOption":5,"SingleOption":1.3184657,"DoubleOption":2.0,"BoolOption":false,"StrOption":"b","Nest":{"IntOption":4}}
1 - {"IntOption":4,"SingleOption":1.8871154,"DoubleOption":-7.0,"BoolOption":true,"StrOption":"b","Nest":{"IntOption":1}}
1 - {"IntOption":4,"SingleOption":1.8871154,"DoubleOption":-7.0,"BoolOption":false,"StrOption":"b","Nest":{"IntOption":1}}
2 - {"IntOption":-2,"SingleOption":5.2511787,"DoubleOption":5.0,"BoolOption":false,"StrOption":"b","Nest":{"IntOption":6}}
2 - {"IntOption":-2,"SingleOption":5.2511787,"DoubleOption":5.0,"BoolOption":false,"StrOption":"b","Nest":{"IntOption":6}}


## Continue learning

> [E2E-Forecasting using SSA with Luna Dataset](./E2E-Forecasting%20using%20SSA%20with%20Luna%20Dataset.ipynb) - This shows how to create `SweepableEstimator` with custom search space.